In [1]:
import numpy as np # for easier computation
from IPython.display import clear_output # to write new outputs on the previous outputs

In [27]:
class Utils:
    def __init__(self,
                 N, M1, M2,
                 alpha_high,
                 alpha_low,
                 alpha_between_irs,
                 noise_std):
        self.N = N
        self.M1 = M1
        self.M2 = M2
        self.alpha_high = alpha_high
        self.alpha_low = alpha_low
        self.alpha_between_irs = alpha_between_irs
        self.noise_std = noise_std
        
    # utils
    def channel_generator(self, 
                           distance,
                           path_loss_exponent,
                           d1, d2):
        return np.matrix((distance ** (-path_loss_exponent/2)) * np.exp(1j * 2*np.pi * np.random.rand(d1, d2)))
    
    def print_results(self,):
        pass

In [23]:
class Antenna():
    def __init__(self, power, utils):
        self.power = power

In [28]:
class User():
    
    def __init__(self,
                 antenna_user_dist,
                 irs1_user_dist,
                 irs2_user_dist,
                 weight,
                 utils):
        self.weight = weight
        self.hsu = utils.channel_generator(distance=antenna_user_dist, # (Antenna--->User) channel 
                                          path_loss_exponent=utils.alpha_high,
                                          d1=1, d2=utils.N)
        self.h1u = utils.channel_generator(distance=irs1_user_dist, # (IRS1--->User) channel 
                                          path_loss_exponent=utils.alpha_low,
                                          d1=1, d2=utils.M1)
        self.h2u = utils.channel_generator(distance=irs2_user_dist, # (IRS2--->User) channel 
                                          path_loss_exponent=utils.alpha_low,
                                          d1=1, d2=utils.M2)
        self.antenna_beam = np.random.rand(utils.N, 1)

In [ ]:
class Irs():
    
    def __init__(self,
                 M,
                 antenna_irs_dist,
                 irs_irs_dist,
                 utils):
        self.Hsi = utils.channel_generator(distance=antenna_irs_dist, # (Antenna--->IRS) channel 
                                          path_loss_exponent=utils.alpha_low,
                                          d1=M, d2=utils.N)
        self.Hii = utils.channel_generator(distance=irs_irs_dist, # (IRS--->IRS) channel 
                                          path_loss_exponent=utils.between_irs,
                                          d1=utils.M2, d2=M)
        self.phase = 2*np.pi * np.random.rand(1, M)[0]
        self.exp_phase = np.exp(1j * self.phase)
        self.diag_exp_phase = np.matrix(np.diag(self.exp_phase))

In [ ]:


# Channels
# Hs1 = np.matrix(np.sqrt(BS_IRS1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(M1, N))) # Antenna - IRS1
# Hs2 = np.matrix(np.sqrt(BS_IRS2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(M2, N))) # Antenna - IRS2

# Between IRS Signal
# H12 = np.matrix(np.zeros((M2, M1))) # IRS1 - IRS2
# H21 = H12 # IRS2 - IRS1

# hsu1 = np.matrix(np.zeros((1, N))) # Blockage Antenna - User1
# hsu1 = np.matrix(np.sqrt(BS_USER1**-ALPHA_HIGH) * np.exp(2*np.pi*1j*np.random.rand(1, N))) # Antenna - User1
# hsu2 = np.matrix(np.sqrt(BS_USER2**-ALPHA_HIGH) * np.exp(2*np.pi*1j*np.random.rand(1, N))) # Antenna - User2

# h1u1 = np.matrix(np.sqrt(IRS1_USER1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M1))) # IRS1 - User1
# h1u2 = np.matrix(np.sqrt(IRS1_USER2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M1))) # IRS1 - User2

# h2u1 = np.matrix(np.zeros((1,M2))) # Blockage IRS2 - User1
# h2u1 = np.matrix(np.sqrt(IRS2_USER1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M2))) # IRS2 - User1
# h2u2 = np.matrix(np.sqrt(IRS2_USER2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M2))) # IRS2 - User2

# Passive Beamforming Coefficients Initialization
# theta1 = np.exp(1j * 2*np.pi * np.random.rand(1, M1)[0])
# theta2 = np.exp(1j * 2*np.pi * np.random.rand(1, M2)[0])
# phi1 = np.matrix(np.diag(theta1))
# phi2 = np.matrix(np.diag(theta2))
# theta1_temp = theta1
# theta2_temp = theta2

# Path for each user
user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

# Active Beamforming Coefficients Initialization
# W1 = np.random.rand(N, 1)
# W2 = np.random.rand(N, 1)
# W1_temp = W1
# W2_temp = W2

In [4]:
class Search_Tools:
    
    def __init__(self,
                 min_search,
                 max_search,
                 epsilon):
        self.min_seach = min_search
        self.max_seach = max_search
        self.epsilon = epsilon
    
    def modified_bisection(self, func,): # Modified Bisection Search Method
        """
        Returns the root of a given function using Modified Bisection Method. In modified bisection method,
        it is assumed that the function is decreasing, f(max_search) < 0 < f(min_search) and
        it has only one root in [min_search, max_search] interval.

            Parameters:
                func (labmda function): purposed labmda function
                max_search (float): upper bound of the search interval
                min_search (float): lower bound of the search interval (default = 0)
                epsilon (float): tolerance level of the root (default = 0.1)

            Returns:
                mid_point (float): root of the function
        """
        min_search = self.min_seach # Because min and max will be changed during the iterations
        max_search = self.max_seach
        while((max_search - min_search) > self.epsilon):
            mid_point = (min_search + max_search)/2
            if(func(mid_point) > 0): # Shorten the interval
                min_search = mid_point
            else:
                max_search = mid_point

        return mid_point

In [29]:
class Solvers:
    
    def __init__(self):
        pass
    
    def coordinate_descent(self):
        pass
    
    def gradient_descent(self):
        pass

In [5]:
class Metrics:
    
    def sinr(self, user_path, W1, W2, sigma): # sinr calculator
        sinr = (np.abs(user_path * W1) ** 2) / (np.abs(user_path * W2) ** 2 + sigma ** 2)
        return sinr

    def rate(self, user_path, W1, W2, sigma): # rate calculator
        sinr = self.sinr(user_path, W1, W2, sigma)
        return np.log2(1 + sinr)

    def sum_rate(self, rate1, rate2, weight1, weight2): # sum-rate calculator
        return (weight1 * rate1) + (weight2 * rate2)
    
    def f1a_calculator(Rate1, Rate2, sinr1, sinr2, w1, w2, alpha1, alpha2):
        return (w1*Rate1 + w2*Rate2 - w1*alpha1 - w2*alpha2 +
                (w1*(1+alpha1)*sinr1/(1+sinr1)) + (w2*(1+alpha2)*sinr2/(1+sinr2)))
        
    def f2a_calculator(temp1, temp2, w1, w2, W1, W2, alpha1, alpha2, b1, b2, sigma):
        return (2*np.sqrt(w1*(1+alpha1))*np.real(np.conjugate(b1)*temp1*W1) +
                2*np.sqrt(w2*(1+alpha2))*np.real(np.conjugate(b2)*temp2*W2) -
                (np.abs(b1)**2)*(np.abs(temp1*W1)**2 + np.abs(temp1*W2)**2 + sigma**2) -
                (np.abs(b2)**2)*(np.abs(temp2*W2)**2 + np.abs(temp2*W1)**2 + sigma**2))
        
    def f3a_calculator(temp1, temp2, w1, w2, W1, W2, alpha1, alpha2, epsilon1, epsilon2, sigma):
        return (2*np.sqrt(w1*(1+alpha1))*np.real(np.conjugate(epsilon1)*temp1*W1) +
                2*np.sqrt(w2*(1+alpha2))*np.real(np.conjugate(epsilon2)*temp2*W2) -
                (np.abs(epsilon1)**2)*(np.abs(temp1*W1)**2 + np.abs(temp1*W2)**2 + sigma**2) -
                (np.abs(epsilon2)**2)*(np.abs(temp2*W2)**2 + np.abs(temp2*W1)**2 + sigma**2))

In [ ]:
# Make sure both IRS have same channel between them

In [ ]:
# N  = 10 # Antenna elements
# M1 = 10 # IRS1 elements
# M2 = 10 # IRS2 elements
# w1 = 1 # First user's weight
# w2 = 1 # Second user's weight
# max_search = 10000000000 # max_search argument for bisection method
# max_power = 0.01 # Transmitted power
# sigma = 0.0001 # Noise standard deviation

# # Path-Loss exponents
# ALPHA_HIGH = 2.5
# ALPHA_LOW  = 2
# BETWEEN_IRS = 2

# # Distances
# BS_IRS1 = 20
# BS_IRS2 = 30
# IRS1_IRS2 = 40
# BS_USER1 = 50
# BS_USER2 = 60
# IRS1_USER1 = 50
# IRS1_USER2 = 40
# IRS2_USER1 = 30
# IRS2_USER2 = 20

In [ ]:
# # remove the double reflection
# H12 = np.matrix(np.sqrt(IRS1_IRS2**-BETWEEN_IRS) * np.exp(2*np.pi*1j*np.random.rand(M2, M1))) # IRS1 - IRS2
# H21 = H12 # IRS2 - IRS1


# theta1 = theta1_temp
# theta2 = theta2_temp
# phi1 = np.matrix(np.diag(theta1))
# phi2 = np.matrix(np.diag(theta2))

# W1 = W1_temp
# W2 = W2_temp

# # Path for each user
# user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
# user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

In [ ]:
# W1 = W1_temp
# W2 = W2_temp

# theta1 = theta1_temp
# theta2 = theta2_temp
# phi1 = np.matrix(np.diag(theta1))
# phi2 = np.matrix(np.diag(theta2))

# print("Theta1 phase: ", np.angle(theta1) * 180 / np.pi)
# print("Theta2 phase: ", np.angle(theta2) * 180 / np.pi)

# # Path for each user
# user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
# user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

In [ ]:
gamma = 0.05

In [ ]:
i = 0
while True:
    # -------------------------------------------------------------------------------------------------------------------------------
    # To remove log2
    sinr1, rate1 = rate(user1_path, W1, W2, sigma)
    sinr2, rate2 = rate(user2_path, W2, W1, sigma)
    alpha1 = sinr1
    alpha2 = sinr2
    # -------------------------------------------------------------------------------------------------------------------------------
    # Active Beamforming Optimization
    # Fraction to Linear
    b1 = (np.sqrt(w1 * (1 + alpha1)) * user1_path * W1) / (np.abs(user1_path * W1)**2 + np.abs(user1_path * W2)**2 + sigma**2)
    b2 = (np.sqrt(w2 * (1 + alpha2)) * user2_path * W2) / (np.abs(user2_path * W2)**2 + np.abs(user2_path * W1)**2 + sigma**2)

    # Lagrangian dual transformation
    temp_func = lambda lambda_temp : (np.linalg.norm(complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_temp * np.eye(N, dtype= "complex128") +
                float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 *user2_path.getH() * user2_path) *
                user1_path.getH())**2 + np.linalg.norm(complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_temp * np.eye(N, dtype= "complex128") +
                float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) *
                user2_path.getH())**2 - max_power)

    # Lagrangian dual variable
    lambda_var = modified_bisection(temp_func, max_search=max_search)

    W1 = (complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_var * np.eye(N, dtype= "complex128") + float(np.abs(b1))**2 * user1_path.getH() *
        user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user1_path.getH())
    W2 = (complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_var * np.eye(N, dtype= "complex128") + float(np.abs(b1))**2 * user1_path.getH() *
        user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user2_path.getH())
    # -------------------------------------------------------------------------------------------------------------------------------
    # Fraction to Linear
    epsilon1 = (np.sqrt(w1*(1+alpha1)) * user1_path*W1) / (np.abs(user1_path*W1)**2 + np.abs(user1_path*W2)**2 + sigma**2)
    epsilon2 = (np.sqrt(w2*(1+alpha2)) * user2_path*W2) / (np.abs(user2_path*W2)**2 + np.abs(user2_path*W1)**2 + sigma**2)

    # IRS1 optimization
    a00 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
                        np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
                        np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W1
    a10 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
                        np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
                        np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W2
    a01 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
                        np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
                        np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W1
    a11 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
                        np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
                        np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W2
    b = np.matrix([[((hsu1 + h2u1*phi2*Hs2)*W1)[0,0],
                    ((hsu2 + h2u2*phi2*Hs2)*W1)[0,0]],
                    [((hsu1 + h2u1*phi2*Hs2)*W2)[0,0],
                    ((hsu2 + h2u2*phi2*Hs2)*W2)[0,0]]])
    U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
        (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
    v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
            (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
        (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
            (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))

    first_part = np.array((-U * np.matrix(theta1).transpose().conjugate() + v).conjugate())
    second_part = np.array((-1j * np.matrix(theta1).transpose().conjugate()))
    derivative = 2 * np.real(first_part * second_part).transpose()[0]
    # gamma = step_size_calculator(theta= theta1, U= U, v= v, gamma_zero= 10000, decay_factor= 0.5, c= 0.0001,
    #                              W1= W1, W2= W2, W1_before= W1_before, W2_before= W2_before,
    #                              t= t, which_theta= 1, phi1= phi1, phi2= phi2)
    phase = np.angle(theta1) + gamma*derivative
    theta1 = np.exp(1j * phase)
    # ------------------------------- #
    phi1 = np.matrix(np.diag(theta1))

    # IRS2 optimization
    a00 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W1
    a10 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W2
    a01 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W1
    a11 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
                        np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W2
    b = np.matrix([[((hsu1 + h1u1*phi1*Hs1)*W1)[0,0],
                    ((hsu2 + h1u2*phi1*Hs1)*W1)[0,0]],
                    [((hsu1 + h1u1*phi1*Hs1)*W2)[0,0],
                    ((hsu2 + h1u2*phi1*Hs1)*W2)[0,0]]])
    U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
        (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
    v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
            (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
        (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
            (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))

    first_part = np.array((-U * np.matrix(theta2).transpose().conjugate() + v).conjugate())
    second_part = np.array((-1j * np.matrix(theta2).transpose().conjugate()))
    derivative = 2 * np.real(first_part * second_part).transpose()[0]
    # gamma = step_size_calculator(theta= theta2, U= U, v= v, gamma_zero= 10000, decay_factor= 0.5, c= 0.0001,
    #                              W1= W1, W2= W2, W1_before= W1_before, W2_before= W2_before,
    #                              t= t, which_theta= 2, phi1= phi1, phi2= phi2)
    phase = np.angle(theta2) + gamma*derivative
    theta2 = np.exp(1j * phase)
    # ------------------------------- #
    phi2 = np.matrix(np.diag(theta2))

    user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
    user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1
# -------------------------------------------------------------------------------------------------------------------------------

    sinr1, rate1 = rate(user1_path, W1, W2, sigma)
    sinr2, rate2 = rate(user2_path, W2, W1, sigma)

    if i % 1 == 0:
        if j % 10 == 0:
            print("Lambda Variable: ", lambda_var)
            print("Ratio Power in percent:", 100 * np.abs(np.linalg.norm(W1)**2 + np.linalg.norm(W2)**2 - max_power)/max_power)
            print("Rate1 : ", rate1[0, 0])
            print("Rate2 : ", rate2[0, 0])
            print("Sum-Rate :", sum_rate(rate1, rate2, w1, w2)[0, 0])
            # print("f3a-Value :", f3a_calculator(user1_path, user2_path, w1, w2, W1, W2, alpha1, alpha2, epsilon1, epsilon2, sigma)[0, 0])
            print("Theta1 phase: ", np.angle(theta1) * 180 / np.pi)
            print("Theta2 phase: ", np.angle(theta2) * 180 / np.pi)
            print("Gamma : ", gamma)
            clear_output(wait=True)

In [ ]:
# # Coordinate Descent
# empty_list1 = []
# i = 0
# # while True:
# for _ in range(400):
#     i += 1
#     # -------------------------------------------------------------------------------------------------------------------------------
#     # To remove log2
#     for _ in range(1):
#         sinr1, rate1 = rate(user1_path, W1, W2, sigma)
#         sinr2, rate2 = rate(user2_path, W2, W1, sigma)
#         alpha1 = sinr1
#         alpha2 = sinr2
#     # -------------------------------------------------------------------------------------------------------------------------------
#         # Active Beamforming Optimization
#     # Fraction to Linear
#     for j in range(1):
#         b1 = (np.sqrt(w1 * (1 + alpha1)) * user1_path * W1) / (np.abs(user1_path * W1)**2 + np.abs(user1_path * W2)**2 + sigma**2)
#         b2 = (np.sqrt(w2 * (1 + alpha2)) * user2_path * W2) / (np.abs(user2_path * W2)**2 + np.abs(user2_path * W1)**2 + sigma**2)

#         # Lagrangian dual transformation
#         temp_func = lambda lambda_temp : (np.linalg.norm(complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_temp * np.eye(N) +
#                     float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 *user2_path.getH() * user2_path) *
#                     user1_path.getH())**2 + np.linalg.norm(complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_temp * np.eye(N) +
#                     float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) *
#                     user2_path.getH())**2 - max_power)

#         # Lagrangian dual variable
#         lambda_var = modified_bisection(temp_func, max_search=max_search)

#         # Antenna beamforming(Active beamforming)
#         W1 = (complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_var * np.eye(N) + float(np.abs(b1))**2 * user1_path.getH() *
#             user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user1_path.getH())
#         W2 = (complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_var * np.eye(N) + float(np.abs(b1))**2 * user1_path.getH() *
#             user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user2_path.getH())
#     # -------------------------------------------------------------------------------------------------------------------------------
#     for j in range(1):
#         # Fraction to Linear
#         epsilon1 = (np.sqrt(w1*(1+alpha1)) * user1_path*W1) / (np.abs(user1_path*W1)**2 + np.abs(user1_path*W2)**2 + sigma**2)
#         epsilon2 = (np.sqrt(w2*(1+alpha2)) * user2_path*W2) / (np.abs(user2_path*W2)**2 + np.abs(user2_path*W1)**2 + sigma**2)

#         # IRS1 optimization
#         a00 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
#                             np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
#                             np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W1
#         a10 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
#                             np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
#                             np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W2
#         a01 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
#                             np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
#                             np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W1
#         a11 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
#                             np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
#                             np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W2
#         b = np.matrix([[((hsu1 + h2u1*phi2*Hs2)*W1)[0,0],
#                         ((hsu2 + h2u2*phi2*Hs2)*W1)[0,0]],
#                         [((hsu1 + h2u1*phi2*Hs2)*W2)[0,0],
#                         ((hsu2 + h2u2*phi2*Hs2)*W2)[0,0]]])
#         U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
#             (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
#         # print(np.real(np.linalg.eigvals(U)))
#         v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
#                 (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
#             (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
#                 (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))
#         # print(theta1)
#         for m1 in range(M1):
#             A1n = U[m1, m1]
#             A2n = v[m1, 0] - ((U[m1, :] @ np.transpose(theta1))[0, 0] - U[m1, m1] * theta1[m1])
#             theta1[m1] = (A2n/np.abs(A2n)) * (np.min((1, np.abs(A2n)/A1n)))
#         print(theta1)

#         # IRS2 optimization
#         a00 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W1
#         a10 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W2
#         a01 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W1
#         a11 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
#                             np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W2
#         b = np.matrix([[((hsu1 + h1u1*phi1*Hs1)*W1)[0,0],
#                         ((hsu2 + h1u2*phi1*Hs1)*W1)[0,0]],
#                         [((hsu1 + h1u1*phi1*Hs1)*W2)[0,0],
#                         ((hsu2 + h1u2*phi1*Hs1)*W2)[0,0]]])
#         U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
#             (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
#         # print(np.real(np.linalg.eigvals(U)))
#         v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
#                 (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
#             (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
#                 (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))
#         for m2 in range(M2):
#             A1n = U[m2, m2]
#             A2n = v[m2, 0] - ((U[m2, :] @ np.transpose(theta2))[0, 0] - U[m2, m2] * theta2[m2])
#             theta2[m2] = (A2n/np.abs(A2n)) * (np.min((1, np.abs(A2n)/A1n)))
#         phi1 = np.matrix(np.diag(theta1.conjugate()))
#         phi2 = np.matrix(np.diag(theta2.conjugate()))

#         user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
#         user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1
#     # -------------------------------------------------------------------------------------------------------------------------------

#         sinr1, rate1 = rate(user1_path, W1, W2, sigma)
#         sinr2, rate2 = rate(user2_path, W2, W1, sigma)

#         if i % 1 == 0:
#             if j % 30 == 0:
#                 # print("i:", i)
#                 print("j:", j)
#                 print("SINR1 : ", sinr1[0, 0])
#                 print("SINR2 : ", sinr2[0, 0])
#                 # print("Rate1 :" , rate1[0, 0])
#                 # print("Rate2 :" , rate2[0, 0])
#                 print("Sum-Rate :", sum_rate(rate1, rate2, w1, w2)[0, 0])
#                 # print("f3a-Value :", f3a_calculator(user1_path, user2_path, w1, w2, W1, W2, alpha1, alpha2, epsilon1, epsilon2, sigma)[0, 0])
#                 print("Theta1 phase: ", np.angle(theta1) * 180 / np.pi)
#                 # print("Theta1 abs: ", np.abs(theta1))
#                 print("Theta2 phase: ", np.angle(theta2) * 180 / np.pi)
#                 # print("Theta2 abs: ", np.abs(theta2))
#                 clear_output(wait=True)

In [ ]:
# quantization_levels_count = 100000
# quantize_levels = np.zeros(quantization_levels_count)
# deference_level = 2*np.pi/quantization_levels_count

# for i in range(1, quantization_levels_count):
#     quantize_levels[i] = quantize_levels[i-1] + deference_level


# # Quantization
# def quantize(quantize_levels, input):
#     while((input < 0) | (input > 2*np.pi)):
#         if(input < 0):
#             input += 2*np.pi
#         else:
#             input -= 2*np.pi
#     return quantize_levels[np.argmin(np.abs(input - quantize_levels))]

In [ ]:
# class Channel:
#     def __init__(self, K, dimensions):
#         self.K = K
#         self.rows = dimensions[0]
#         self.columns = dimensions[1]

#     def los_component(self):
#         los_amplitude = np.sqrt(self.K/(self.K + 1))  # Amplitude of LOS component
#         los_phase = np.random.uniform(0, 2*np.pi, (self.rows, self.columns))  # Phase of LOS component
#         los_signal = los_amplitude*np.exp(1j*los_phase)  # LOS signal
#         return los_signal

#     def scattered_component(self):
#         scattered_variance = 1/(self.K + 1)  # Variance of scattered component
#         scattered_signal = np.sqrt(scattered_variance) * np.sqrt(1/2) * (np.random.randn(self.rows, self.columns) +
#                                                           1j*np.random.randn(self.rows, self.columns))
#         return scattered_signal

#     def make_rician(self):
#         return np.matrix(self.los_component() + self.scattered_component())
    
    
# # Channel Model 2

# K = 5 # Rician factor

# # Channels
# Hs1 = Channel(K, (M1, N)).make_rician()
# Hs2 = Channel(K, (M2, N)).make_rician()

# # Between IRS Signal
# H12 = Channel(K, (M2, M1)).make_rician()
# # H12 = np.matrix(np.zeros((M2, M1)))
# H21 = H12.getH()

# # hsu1 = np.matrix(np.zeros((1, N))) # Blockage
# hsu1 = Channel(K, (1, N)).make_rician()

# hsu2 = Channel(K, (1, N)).make_rician()

# # h1u1 = np.matrix(np.zeros((1, M1))) # Blockage
# h1u1 = Channel(K, (1, M1)).make_rician()

# h1u2 = Channel(K, (1, M1)).make_rician()

# # h2u1 = np.matrix(np.zeros((1,M2))) # Blockage
# h2u1 = Channel(K, (1, M2)).make_rician()

# h2u2 = Channel(K, (1, M2)).make_rician()

In [ ]:
# def step_size_calculator(theta, U, v, gamma_zero, decay_factor, c, W1, W2, W1_before, W2_before, t, which_theta, phi1, phi2):
#     gamma = gamma_zero
#     if t != 0:
#         while(True):
#             phi_before = np.matrix(np.diag(theta.conjugate()))

#             first_part = np.array((-U * np.matrix(theta).transpose() + v).conjugate())
#             second_part = np.array((-1j * np.matrix(theta).transpose()))
#             derivative = 2 * np.real(first_part * second_part).transpose()[0]
#             phase = np.angle(theta) + gamma*derivative
#             theta = np.exp(1j * phase)
#             phi = np.matrix(np.diag(theta))

#             if which_theta == 1:
#                 # phi2 = phi2

#                 phi1 = phi_before
#                 user1_path_before = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
#                 user2_path_before = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

#                 phi1 = phi
#                 user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
#                 user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

#             elif which_theta == 2:
#                 # phi1 = phi1

#                 phi2 = phi_before
#                 user1_path_before = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
#                 user2_path_before = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

#                 phi2 = phi
#                 user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
#                 user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

#             [_, _, rate1, rate2] = Rate_Calculator(user1_path, user2_path, W1, W2, sigma)
#             sum_rate = f1_calculator(rate1, rate2, w1, w2)[0, 0]

#             [_, _, rate1_before, rate2_before] = Rate_Calculator(user1_path_before, user2_path_before, W1_before, W2_before, sigma)
#             sum_rate_before = f1_calculator(rate1_before, rate2_before, w1, w2)[0, 0]

#             if sum_rate < (sum_rate_before + c * gamma * (np.linalg.norm(derivative) ** 2)):
#                 gamma *= decay_factor
#                 if(gamma <= 1e-5):
#                     return gamma
#             else:
#                 return gamma

#     else:
#         return gamma